In [1]:
import pandas as pd
import os
import sys

In [2]:
from pipeline import Pipeline
from devgpt_pipeline.extract.extractdata import DataExtractor

pipeline = Pipeline()
extract_component = DataExtractor()
pipeline.add_component(extract_component)
pipeline.run()


Running Extract
Completed Extract


'Completed'

In [2]:
# get the table name of the devgpt.sqlite
from sqlalchemy import create_engine
from sqlalchemy import text
# import sessio
from sqlalchemy.orm import sessionmaker

engine = create_engine('sqlite:///devgpt.sqlite')
Session = sessionmaker(bind=engine)
session = Session()
table_name = session.execute(text("SELECT name FROM sqlite_master WHERE type='table'")).fetchall()
print(table_name)


[]


In [3]:


folder = "extracted"
data_files = []
for file in os.listdir(folder):
    if file.endswith(".csv"):
        data_files.append(os.path.join(folder, file))

extraction = pd.concat((pd.read_csv(f) for f in data_files))
extraction = extraction.drop_duplicates(subset=['uid','snapshot'])
print(f"The pipeline extracted {len(extraction)} rows of data")


The pipeline extracted 64146 rows of data


In [4]:
# from pipeline import Pipeline
# from devgpt_pipeline.extract.extractdata import DataExtractor
# from devgpt_pipeline.load.data_loader import DataLoader
# pipeline = Pipeline()
# Load_component = DataLoader()
# pipeline.add_component(Load_component)
# pipeline.run()


Running Load
Loading data...
Tables cleared
9 snapshots loaded
1690 pull requests loaded
3079 issues loaded
1633 hacker news loaded
398 discussions loaded
2969 commits loaded
11826 sharings loaded
42551 conversations loaded
Data loaded
Completed Load


'Completed'

In [ ]:
# from devgpt_pipeline.translator.translator import LanguageTranslatorComponent
# pipeline = Pipeline()
# lang_translate = LanguageTranslatorComponent()
# pipeline.add_component(lang_translate)
# pipeline.run()


In [1]:
# count of non-english rows by table
from devgpt_pipeline.load.model import Conversation, Issue, PullRequest, Commit, Sharing, Discussion, HackerNews
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy import func
from sqlalchemy import distinct
from sqlalchemy import and_


engine = create_engine('sqlite:///devgpt.sqlite')
Session = sessionmaker(bind=engine)
session = Session()

# count of non-english rows by table, look at the language_id column
print("Count of non-english rows by table")
print(f"Conversation: {session.query(Conversation).filter(Conversation.language_id != 'en').count()}")
print(f"Translated Conversation: {session.query(Conversation).filter(Conversation.language == 'translated').count()}")
print(f"Issue: {session.query(Issue).filter(Issue.language_id != 'en').count()}")
print(f"PullRequest: {session.query(PullRequest).filter(PullRequest.language_id != 'en').count()}")
print(f"Commit: {session.query(Commit).filter(Commit.language_id != 'en').count()}")
print(f"Sharing: {session.query(Sharing).filter(Sharing.language_id != 'en').count()}")
print(f"Discussion: {session.query(Discussion).filter(Discussion.language_id != 'en').count()}")
print("")

# get their ids
non_english_dict = {}
non_english_dict['Conversation'] = [x.id for x in session.query(Conversation).filter(Conversation.language_id != 'en').all()]
non_english_dict['Issue'] = [x.id for x in session.query(Issue).filter(Issue.language_id != 'en').all()]
non_english_dict['PullRequest'] = [x.id for x in session.query(PullRequest).filter(PullRequest.language_id != 'en').all()]
non_english_dict['Commit'] = [x.id for x in session.query(Commit).filter(Commit.language_id != 'en').all()]
non_english_dict['Sharing'] = [x.id for x in session.query(Sharing).filter(Sharing.language_id != 'en').all()]
non_english_dict['Discussion'] = [x.id for x in session.query(Discussion).filter(Discussion.language_id != 'en').all()]

# get a list of 1000 Prompts from Conversation




Count of non-english rows by table
Conversation: 4703
Translated Conversation: 4703
Issue: 0
PullRequest: 0
Commit: 0
Sharing: 0
Discussion: 0



In [6]:
# 3000 prompts unique by id ( the table has non unique ids)
from sqlalchemy import distinct

prompts = []
prompts = prompts = (
    session.query(Conversation.Prompt, Conversation.id)
    .group_by(Conversation.Prompt)
    .order_by(Conversation.id)  # You can adjust the ordering as needed
    .all()
)
# 3000 prompts
prompts = prompts[:3000]
prompts = [x[0] for x in prompts]
# save it as a csv
import pandas as pd
df = pd.DataFrame(prompts, columns=['prompt'])
df.to_csv('prompts.csv', index=False)



In [9]:
import pandas as pd
conv_ids = non_english_dict['Conversation']
matches = pd.read_csv('intent/overall/keywords_conversation.csv')
matches['id'].tolist()
print(matches['id'].tolist())



[4814289010674082962, 4814289010674082962, 5381074618616449285, 5381074618616449285, -3404078680426662067, -3404078680426662067, 8442416888609976579, -7867832552257311048, -3047351236530904524, -7002928619944232992, 5839603502237537554, 5839603502237537554, 5063305002769897312, -7028267975837638473, -7028267975837638473, -6000152008868623652, -6000152008868623652, 5165064244739701628, 5165064244739701628, 1607700620044220263, 1607700620044220263, -3003364634313346779, -3003364634313346779, 8117472922205679955, 8117472922205679955, -6314574530624725446, -6314574530624725446, 8745803038446745720, 8745803038446745720, 848272366974399469, 848272366974399469, -3080835856101482765, -1049209901123960985, -8940141726826073835, 5165064244739701628, 5165064244739701628, 1690059209462372715, -5177304715430282565, 4744690687787489291, 8695008064040069208, -7014860062863161696, -9026663766300063345, 5768708304635389816, 5768708304635389816, 2551400555139168165, 848272366974399469, 84827236697439946

In [4]:
non_empty_promopt_connt = session.query(Conversation).filter(Conversation.Prompt != None).count()
non_empty_promopt_connt



42505

In [6]:
noon_empty_response_count = session.query(Conversation).filter(Conversation.Answer != None).count()
noon_empty_response_count

42551